In [ ]:
import pandas as pd

df = pd.read_csv('Ecommerce_data.csv')

print(df)


In [ ]:
import pandas as pd

df2 = pd.read_csv('gender.csv')

print(df2)

In [ ]:
from dateutil import parser
# Function to parse and convert dates
def parse_date(date_str):
    try:
        return parser.parse(date_str).strftime('%m-%d-%y')
    except (parser.ParserError, TypeError, ValueError):
        return None

# Apply the function to the 'order_date' column
df['order_date'] = df['order_date'].apply(parse_date)

print("\nDataFrame with Normalized Date Formats:")
print(df)
def fill_missing_with_mean(df):
    # Iterate over each column in the DataFrame
    for column in df.columns:
        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(df[column]):
            # Calculate the mean of the column
            mean_value = df[column].mean()
            # Replace missing values with the mean
            df[column].fillna(mean_value, inplace=True)
    return df
# Function to limit the number of digits after the decimal point and make an approximation
def round_columns_to_n_digits(df, columns, n):
    for column in columns:
        df[column] = df[column].round(n)
    return df
df = round_columns_to_n_digits(df, ['sales_per_order', 'order_item_discount', 'profit_per_order'], 2)

print("\nDataFrame after Rounding Values:")
print(df)
import numpy as np

# Function to identify outliers in all columns using the IQR method
def find_outliers_all_columns(df):

    outliers_dict = {}
    
    for column in df.select_dtypes(include=[np.number]).columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        
        # Calculate IQR (Interquartile Range)
        IQR = Q3 - Q1
        
        # Define the bounds for identifying outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Find the outliers
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        
        # Add the outliers DataFrame to the dictionary
        outliers_dict[column] = outliers
        # Replace outliers with the mean
        df[column] = np.where((df[column] < lower_bound) | (df[column] > upper_bound), mean_value, df[column])

    return outliers_dict

print(df)

In [ ]:
# Create a dictionary from the names_gender dataset for quick lookup
names_gender_dict = dict(zip(df2['Name'], df2['Gender']))

# Add a new column 'gender' to the original dataset by mapping the names to the gender dictionary
df['Gender'] = df['customer_first_name'].map(names_gender_dict)
df['Gender'] = df['Gender'].apply(lambda x: 'M' if x == 'F' else 'F' if x == 'M' else x)
print(df)
# Save the DataFrame to a CSV file
df.to_csv('cleaned_ecommerce_dataset.csv', index=False)
print(df)

In [ ]:
import json
from datetime import datetime
with open('holidays.json', 'r') as f:
  holiday_data = json.load(f)
# Change the date format to mm-dd-yy
for entry in holiday_data:
    entry['date'] = datetime.strptime(entry['date'], '%Y-%m-%d').strftime('%m-%d-%y')
# Extract the list of holiday dates from the JSON dataset
holiday_dates = [entry['date'] for entry in holiday_data]

# Create a new column 'isholiday' in the CSV dataset
df['isholiday'] = df['order_date'].apply(lambda x: 'yes' if x in holiday_dates else 'no')
df3 = pd.read_csv('cleaned_data_final.csv')
columns_to_delete = ['ship_date', 'days_for_shipment_real' , 'customer_country','days_for_shipment_scheduled']
df3.drop(columns=columns_to_delete, inplace=True)
print(df3)


In [ ]:
import pandas as pd 
df_final=pd.read_csv('cleaned_data_final2.csv',encoding='utf-8')
pd.set_option('display.max_columns', None)
print(df_final)
import snowflake.connector
import logging

# Enable logging for Snowflake connector
logger = logging.getLogger('snowflake.connector')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

# Snowflake connection setup
conn = snowflake.connector.connect(
    user="khalilsamet",
    password="********",
    account="USPEGSJ-UT92666", 
    warehouse="SHOPIFY_WAREHOUSE",
    database="SHOPIFY_DATABASE",
    schema="SHOPIFY_SCHEMA"
)

print("Connection successful!")
df_products = df_final[['product_id', 'product_name', 'category_name']]
df_customers = df_final[['customer_id', 'customer_first_name','customer_last_name','customer_segment','Gender','location_id']]
df_time = df_final[['time_id', 'order_date','isholiday']]
df_location = df_final[['location_id', 'customer_city','customer_state','customer_region']]
df_shipment = df_final[['ship_id', 'shipping_type','delivery_status']]
df_sales = df_final[['order_id', 'customer_id','time_id','ship_id','product_id','order_quantity','sales_per_order','profit_per_order','order_item_discount']]
def check_column_data_types(df):
    print(f"\n=== Data Types in {df} ===")
    column_data_types = df.dtypes
    type_summary = pd.DataFrame({
        'Column Name': column_data_types.index,
        'Data Type': column_data_types.values
    })
    print(type_summary)
    return type_summary

check_column_data_types(df_products)
check_column_data_types(df_shipment)
check_column_data_types(df_location)
check_column_data_types(df_customers)
check_column_data_types(df_time)
check_column_data_types(df_sales)


In [ ]:
df_products = df_final[['product_id', 'product_name', 'category_name']]
df_customers = df_final[['customer_id', 'customer_first_name','customer_last_name','customer_segment','Gender','location_id']]
df_time = df_final[['time_id', 'order_date','isholiday']]
df_location = df_final[['location_id', 'customer_city','customer_state','customer_region']]
df_shipment = df_final[['ship_id', 'shipping_type','delivery_status']]
df_sales = df_final[['order_id', 'customer_id','time_id','ship_id','product_id','order_quantity','sales_per_order','profit_per_order','order_item_discount']]
def check_column_data_types(df):
    print(f"\n=== Data Types in {df} ===")
    column_data_types = df.dtypes
    type_summary = pd.DataFrame({
        'Column Name': column_data_types.index,
        'Data Type': column_data_types.values
    })
    print(type_summary)
    return type_summary

check_column_data_types(df_products)
check_column_data_types(df_shipment)
check_column_data_types(df_location)
check_column_data_types(df_customers)
df_time['order_date'] = pd.to_datetime(df_time['order_date'], format='%m-%d-%y')
check_column_data_types(df_time)
check_column_data_types(df_sales)



In [ ]:
def insert_data(table_name, dataframe):
    for _, row in dataframe.iterrows():
        placeholders = ', '.join(['%s'] * len(row))
        query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        conn.cursor().execute(query, tuple(row))

# Insert data into Snowflake tables
insert_data("PRODUCTS_DIM", df_products)
insert_data("TIME_DIM", df_time)
insert_data("LOCATION_DIM", df_location)
insert_data("CUSTOMER_DIM", df_customers)
insert_data("SHIPMENT_DIM", df_shipment)
insert_data("SALES_FACT", df_sales)
print("Data loaded successfully!")
print(df_products)
